In [ ]:
from dotenv import load_dotenv
from pymongo import MongoClient
import os
import pandas as pd

os.chdir('../')
os.getcwd()

'c:\\Users\\11\\Desktop\\LCG\\Programming\\project\\wedrive'

In [ ]:
# 데이터베이스 연결
load_dotenv()
client = MongoClient(os.getenv('DB_ADR'),
          username=os.getenv('DB_USER'),
          password=os.getenv('DB_PASSWORD'),
          authSource=os.getenv('DB_AuthSource'),
          authMechanism=os.getenv('DB_AuthMechanism'))

db = client.get_database(os.getenv('DB_Collection'))

cl_2020 = db.get_collection('raw_od_uuid_2020')
cl_2021 = db.get_collection('raw_od_uuid_2021')
cl_2022 = db.get_collection('raw_od_uuid_2022')
cl_2023 = db.get_collection('raw_od_uuid_2023')
cl_2024 = db.get_collection('raw_od_uuid_2024')

In [ ]:
# 2020, 2021, 2022, 2023, 2024 컬렉션에서 uuid set으로 저장
uuid_2020_set=set(cl_2020.distinct("uuid"))
uuid_2021_set=set(cl_2021.distinct("uuid"))
uuid_2022_set=set(cl_2022.distinct("uuid"))
uuid_2023_set=set(cl_2023.distinct("uuid"))
uuid_2024_set=set(cl_2024.distinct("uuid"))

In [ ]:
# 겹치는 아이디 카운팅
common_uuid = uuid_2020_set & uuid_2021_set & uuid_2022_set & uuid_2023_set & uuid_2024_set
print(f'20~24년 기간 겹치는 아이디 개수: {len(common_uuid)}개')

# 겹치는 아이디 csv 파일로 저장
common_uuid_list=pd.DataFrame(common_uuid, columns=['uuid'])
common_uuid_list.to_csv(os.getcwd()+'/20~24년_공통_uuid.csv')

In [14]:
common_uuid=pd.read_csv(os.getcwd()+'/data/20~24년_공통_uuid.csv', index_col=False)
common_uuid_list=list(common_uuid['uuid'])
common_uuid_list

['166678e54218474090d6a9f47efa117f',
 'd4685d93c0964880bf03297699761232',
 '3b3e5d25923b42b28fdbcd853be5f301',
 '681424b8665b47e59144038a4e95c494',
 'fc2543fa6c90483085e4fd32db06f49e',
 'c0eb99e17e7e4c7787ef374db17c995c',
 'ffc5a438782c4f5ca6c9ae740221f220',
 '3efe9582a68a47e1a37a64caad87f77a',
 'e43b8e29c73d4a88b1a6726e2f18a836',
 '5f5f55c78dd14e728aded2253a44516d',
 'c7be2c4315ee475e85f6d2eccbb76580',
 'c379833ee1854f39a3be86d0b8d31391',
 '36e64dcf55874d459c2dcf876ef687e0',
 '759c0661aaa345249e623cd5fffee9db',
 '2e07554666b7411f91b4195125bcb13c',
 'bfaf949fdd3a4400993530758a991874',
 '5c744df868ce4acfae00246eb0cc51bf',
 '189aa2be31704d0cb3212ce108e1eca4',
 '18ecf688336d4f1cb356bf0896292289',
 '887f8d8c3a9248d0939eb9696bf78355',
 '388083fae4ce48ebac0a263f2cd105f6',
 '202b8eeb6e8a4e9cb2e99cbdc88a32f7',
 '4d1210857f914da694b05267724a6cd6',
 '9cc6473316fe4310abe0caa1c63fe38e',
 '41b5d8fbe52a4c40850a912d766a8caa',
 '50b51a5076a24135afbeea736b56606d',
 '6cf36f2fa77b4c0188d0fd4d64b8114d',
 

In [16]:
# timefiltered 데이터에서 common_list에 있는 데이터만 추출해서 디비에 인서트
cl_list=['timefiltered_2023','timefiltered_2022', 'timefiltered_2021', 'timefiltered_2020']
cl_raw_test_1=db.get_collection('raw_test_1')

for cl in cl_list:
    cl_timefiltered = db.get_collection(cl)
    matching_docs=cl_timefiltered.find({"uuid":{"$in":common_uuid_list}})
    if matching_docs:
        cl_raw_test_1.insert_many(matching_docs)
        print(f"{cl}컬렉션에서 {cl_raw_test_1.count_documents({})}개의 문서가 추가되었습니다.")
    else:
        print("일치하는 문서가 없습니다.")

timefiltered_2023컬렉션에서 1998055개의 문서가 추가되었습니다.
timefiltered_2022컬렉션에서 4261915개의 문서가 추가되었습니다.
timefiltered_2021컬렉션에서 6650999개의 문서가 추가되었습니다.
timefiltered_2020컬렉션에서 8475695개의 문서가 추가되었습니다.
